<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/encrypting_config_components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [ ]:
!pip install "ipython>=7"

# Set Up Environment

### Import required libraries

In [ ]:
!pip install -q canopy

In [2]:
import canopy
import logging
import nest_asyncio

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()

### Authenticate

In [ ]:
authentication_data = canopy.prompt_for_authentication()
session = canopy.Session(authentication_data)

# Example: Encrypting config components

## Encrypting arbitrary data

The API exposes endpoints for encrypting and decrypting any JSON structure.

We can access these endpoints using the OpenAPI generated wrappers in the Canopy Python library.

In the example below we encrypt some arbitrary data:

In [14]:
# Created the OpenAPI generated wrapper for the config APIs.
config_api = canopy.openapi.ConfigApi(session.async_client)

# Create some arbitrary data.
initial_data = { 'hello': 'world' }

# Encrypt the data using the API.
encrypted_data = await config_api.config_encrypt(initial_data)

print(f'Encrypted: {encrypted_data}')

Encrypted: {'name': 'encrypted', 'metadata': 'jLFfe44JHJrf940B6LBjpwj5xI/LD3eCjtx90uGxgJj0zHN8PRNQkpKZhUNvuSdF7nnKqn9YXOb8xpn5c6JqdjrdAegC8i8r0eIEYpr6UtVNvYxwyk6RTabQZCqezR0y8Pxrxg6x72aLaAnXO8G9Zlp1xo9H7vthBo9Hxxl4BbCtt6IA2V3XI98GxxDlY8EWRgeVguiIxV9DE4/EcPJ6dA==', 'data': '4kMsTCxgbZFAPKFKRHWVA4tu0KtCztnnNxfezlrECdBvlfIn0s47sSNi+uC0I0A+'}


We can then decrypt it:

In [15]:
decrypted_data = await config_api.config_decrypt(encrypted_data)

print(f'Decrypted: {decrypted_data}')

Decrypted: {'hello': 'world'}


**Encrypted data can only be decrypted by users within the same Canopy tenant (users who enter the same "company" name when logging in to the Canopy platform).**

Users in other tenants can use the encrypted data in their configs, but cannot decrypt it themselves.

This makes it a secure way to share car components with other Canopy tenants for simulation, without them being able to see the parameters. We can also hide sensitive output channels when certain areas of the car are encrypted.

## Encrypting part of a config

Only certain parts of a config can be encrypted, as the encrypted version must still conform to the schema. You can contact Canopy to discuss your encryption requirements.

In this example we will encrypt the electric motors of a car, which is an area that has already been enabled for encryption.

First we will fetch a default car and output its `electricMotors` section.

In [16]:
default_car = await canopy.load_default_config(session, 'car', 'Canopy F1 Car 2019')

print(default_car.data.powertrain.electric.electricMotors)

[Munch({'name': 'ERSMotor', 'deployment': Munch({'name': 'eMotor with Constant Power', 'PMotorMax': 120000, 'eMotor': 0.95, 'PHarvestRegulatoryLimit': 120000}), 'torqueApplicationPosition': [Munch({'position': 'Rear crankshaft', 'rApplication': 1})], 'transmissionConnection': 'Primary', 'harvestIndependence': Munch({'name': 'Achieve Target Brake Balance'}), 'motorController': 'Rear axle', 'IMotor': 0, 'PPartThrottleHarvest': 30000, 'rPartThrottleChargeOn': 0.2, 'rPartThrottleChargeOff': 0.65, 'rElectricMotorPowerFactor': 1, 'PElectricMotorPowerOffset': 0, 'PMotorMeasurementPosition': 'Motor output shaft'})]


Next we will encrypt the `electricMotors` section.

In [17]:
encryption_result = await config_api.config_encrypt(default_car.data.powertrain.electric.electricMotors)

print(encryption_result)

{'name': 'encrypted', 'metadata': 'Eoy8pF9SelRlw5VSeNtKg0IvFzM0knjYfzXmAxnEvh8KezKuv6MV4DsIzET5mxxBtivD+uFJsSBdv8OTYUYpxyjeT4e1tiFE6VzTXjyerDb7yNb4ch7h6Ua/Ri3DOWYULKmoGZjrbhMQ/tgOSEOPjEalnm1cJ7RD8YICAMwxIbAp9hKX6OlhMCOMDtJ14kOxJaXz+BWIwX/Jl77hc/3P4g==', 'data': '4Hp5TNr//M2CsNGbl6dWy4kIJg7T6tCUFlifJCqcH+SIiQ8xJ4pWhyw9EarFzy8R9iyAi9yyNkfP2eIQI5jTQMG7knUhz/SKUChSfg7E/eV2Y5ELvTc6+yT3l3+4DRbG4fD6bq6Qtyg+LRnOG/f3huVqzFhoiW1vqcZt+LfxjYAqsK7aFXZlM0rNNsnwgmKI5/CEnZe+QGLDblHbKROpRnKpKnwrcmG1I880zQuEB0ea178OmdoPRid1JPPPsH84zyUtIUGclSxFSKuDhv5l+khqO1/EtLJ4kNW5Rp7O9T1iSDR7OUKAYlkHdIdIp/eRQFwdPwAWKzmtBHxfM1vd3SQoUasTNQ6rmd7xBPRLxIynYyYMB97S1Pb+QXfMatGhAWRUDixOygM/GNJmB2Us3QebKMyR5VSbISxCP7WRNCaqpfV5cucBzrH9F0zsqoONlBMDiS6IXdqtFCnbIgJy9R5efDjYklX+VJEoO0eRwVOvSYFsyTnVWA6bGeyQz8o/mWq8BUrx4d3XIlGlUMuBCxoQlavY/m/6CeKIKSUx+4rzB+umhp0cli8ck1LIVBk9/AIui37hbcKFpG/cyGw1oymnKIJeS/XJ23GtI5i4jZ6UrIbh7gIu6QvHUTIkQcpma4HZHH7BHimdSND+Kwwrk5walj2B9HtfPeHU0mURdlyqTxXhhVgjznoLM/OjIGo/vSOC/QYzcQIx0suKEDUFcjekzMq3QyYb5

Next we will replace the `electricMotors` section of the car with the encrypted version and save the car to the platform.

In [18]:
default_car.data.powertrain.electric.electricMotors = encryption_result

user_car_id = await canopy.create_config(
    session,
    'car',
    'Python Example - Car with encrypted electric motors',
    default_car.raw_data)

print(user_car_id)

468842a9200542aaa21223817954a5f2


And finally we will load the car again and output the `electricMotors` section, to show that it has been saved encrypted.

In [19]:
user_car = await canopy.load_config(session, user_car_id)

print(user_car.data.powertrain.electric.electricMotors)

Munch({'name': 'encrypted', 'metadata': 'LQbiZgMTkWCrykz3gM6LVeFasTmxyDuLtOv3O8D9wlHjfsmymOmFOfFzq1O2M2FHp/ynk8tnXg39Ybqx1f75XtcZq9LxbiQ/Eb9opVnHPxV7FAyVAXEvAmaI5bNWC+RfyoBtBu8ElAsgglA4f6dkupM2htILMxzFpQOlWqEf4QL99skZwyzMViF1lbh0mLEkezoSrMHr5Zib3w8Hx/06mw==', 'data': 'H1gCaPiWXBQ79lyMD9bltPkcEJURW5LyjiY627YnahPxzXsoyBYHTMCRA+N+3ErER4lYaXAy2xEu24XZqDrrOOV5lwuLM+Xv3VvDHkci3QhQPWmZ2pfK6Sw2c1BAvhL5CCGgCHGVT7xcfdulD7lOAK8iFt1wdszSL+5/YSvJcGtqRUmdGjVUbu3++Fxn9z5mnqmUBQHKBxYtVKqTSfk5agfJJoDEp0Vbwlbq1GpZpSXrpI7EK0hLs8VCXc45VPKJNCOMcayh/BpAAblZT+oHnj9295SLM0ZuaZg8QYuXcFKRo22hXNa/HJ5bjddHS1H2nNhLrPbZdJkZ4UElLkf93bHH9IZnRnZthC3QrWeAlVR/NH8qu7KpETQrbRof9uUuz+iKoLtjQQaJ5tOxaYFsUl09yS5yPp2v5QnzSzFSW53k6kLKy5GixURvWMnce/Fb0UOEFZGNytfNx8kAG9wioFqrFTnEMinYLC06pak/E/Lcqpx3aFx8lk19UBrXJUzuW8YL6TZvOJfof19qQzlPoBfVo+mV8ueP2FUUen8JgF6Hoy99IFSqy5FzMxOARGRW7zSEsz+y2FVrz/PU/cpNjpeUQqO/v1JhWchOQldmIK7GKIn+VOaLgtUCMRYWOJ2p2E6ZzuH+1kw9ciOfufDiZEuuV0vko0qjsouqGjLy5QcyPG9btXoNo/rGBJUNikfJGY6Ks8dl0dGiMOcL4JSrlzzKYB1